In [2]:
# K-means
import numpy as np
from sklearn import cluster, datasets
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import pickle
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials


In [3]:
secrets_file = open("secrets-Copy1.txt","r")

In [4]:
string = secrets_file.read()

In [5]:
string.split('\n')

['',
 '',
 'cid:03cc2e02247b4b9ca1f21cb46e59ce47',
 'csecret:f938e4f2bdb04691a103bbcac8b21710']

In [6]:
secrets_dict={}
for line in string.split('\n'):
    if len(line) > 0:
        secrets_dict[line.split(':')[0]]=line.split(':')[1].strip()

In [7]:

#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['cid'],
                                                           client_secret=secrets_dict['csecret']))

In [8]:
top100songs=pd.read_csv('new_data.csv')

In [9]:
recommend_df=pd.read_csv("recommend_df.csv")

In [10]:
scaler= pickle.load(open('scaler.pkl','rb'))

In [11]:
Kmeans= pickle.load(open('kmean.pkl', 'rb'))

In [12]:
import random
x = random.randint(0,100)
song_title = input ('please insert song name:')
song_artist = input('please insert artist:')

# check if the song is in the DataFrame
if song_title in top100songs['title'].values:
    # if the song is in the DataFrame, recommend a different song
    recommended_song = top100songs.loc[top100songs['title'] != song_title, 'title'].iloc[x]
    print("You might like the song:", recommended_song)
else:
    # if the song is not in the DataFrame, print a message
    print("Sorry, we cannot recommend a song.")
if song_artist in top100songs['artist'].values:
    recommended_artist = top100songs.loc[top100songs['artist'] != song_artist, 'artist'].iloc[x]
    print ('You might also like this artist:', recommended_artist)
else: 
    print("Sorry,we cannot recommend an artist.")

please insert song name:Flowers
please insert artist:Miley Cyrus
You might like the song: Escapism.
You might also like this artist: Sam Smith & Kim Petras


In [13]:
def features(track, artist):
    track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')
    uri = track_id["tracks"]["items"][0]['id']
    features = sp.audio_features(uri)
    return features

In [14]:
new_audio= features(song_title,song_artist)

In [15]:
dict_audio= new_audio[0]
dict_audio

{'danceability': 0.707,
 'energy': 0.681,
 'key': 0,
 'loudness': -4.325,
 'mode': 1,
 'speechiness': 0.0668,
 'acousticness': 0.0632,
 'instrumentalness': 5.15e-06,
 'liveness': 0.0322,
 'valence': 0.646,
 'tempo': 117.999,
 'type': 'audio_features',
 'id': '0yLdNVWF3Srea0uzk55zFn',
 'uri': 'spotify:track:0yLdNVWF3Srea0uzk55zFn',
 'track_href': 'https://api.spotify.com/v1/tracks/0yLdNVWF3Srea0uzk55zFn',
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0yLdNVWF3Srea0uzk55zFn',
 'duration_ms': 200455,
 'time_signature': 4}

In [16]:
df= pd.DataFrame.from_dict(dict_audio,orient="index").T

In [17]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,audio_features,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4


In [18]:
df.drop(['type', 'id', 'uri', 'track_href','analysis_url'],axis=1, inplace= True)
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,200455,4


In [19]:
df_scaled= scaler.transform(df)

In [20]:
df_scaled

array([[ 0.83802292,  0.13039216, -1.45767689,  0.75895597,  0.72712269,
        -0.03555236, -0.60368238, -0.40302986, -0.99308441,  0.69502678,
        -0.08740237, -0.2246338 ,  0.18522809]])

In [21]:
# Predicting / assigning the clusters:
new_cluster = Kmeans.predict(df_scaled)

# Check the size of the clusters
pd.Series(clusters).value_counts().sort_index()
new_cluster[0]

/Users/nancyomozokpea/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


NameError: name 'clusters' is not defined

In [22]:
df_2= recommend_df[recommend_df["Clusters"]==new_cluster[0]]

In [29]:
import random
random.choice(top100songs)  
random.randint(0, len(top100songs))
song_title = input ('please insert song name:')
song_artist = input('please insert artist:')

track_id = sp.search(q='artist:' + song_artist + ' track:' + song_title, type='track')
if len(track_id['tracks']['items']) > 0:
    uri = track_id["tracks"]["items"][0]['id']
    features = sp.audio_features(uri)
    # process the features
else:
    recommended_song = random.choice(df_2['song'])
    print("We couldn't find the song '{song_title}' by {song_artist}. Here's a recommended song: {recommended_song}")

if len(sp.search(q='artist:' + song_artist, type='artist')['artists']['items']) > 0:
    uri = track_id["tracks"]["items"][0]['id']
    features = sp.audio_features(uri)
    # process the features
else:
    recommended_artist = random.choice(df_2['artist'])
    print(f"We couldn't find any songs by {song_artist}. Here's a recommended artist: {recommended_artist}")


KeyError: 23